In [ ]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install scikit-learn joblib


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string

In [ ]:
fake = pd.read_csv("Fake.csv")
true = pd.read_csv("True.csv")

In [ ]:
fake.head()
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
fake["class"] = 0
true["class"] = 1

In [ ]:
data = pd.concat([fake, true], axis=0)

In [ ]:
data.sample(10)

,title,text,subject,date,class
5705,U.S. Republicans wrestle with effort to gut Ob...,WASHINGTON (Reuters) - Republicans in Congress...,politicsNews,"February 2, 2017",1
18805,U.N. agrees new team of experts for Burundi bu...,GENEVA (Reuters) - The U.N. Human Rights Counc...,worldnews,"September 28, 2017",1
2275,"Trump on Twitter (Aug 7): Fake News, Senator R...",The following statements were posted to the ve...,politicsNews,"August 7, 2017",1
2618,Coretta Scott King’s Daughter Speaks Up And A...,When it comes to strategizing an offense again...,News,"February 8, 2017",0
23180,WARNING: Don’t Worry If the Stock Market Goes ...,BloombergIn the hours after the president is ...,Middle-east,"November 7, 2016",0
18772,U.S. air strike kills 'several' Islamic State ...,WASHINGTON (Reuters) - Two U.S. air strikes ki...,worldnews,"September 28, 2017",1
5162,New Polls Stun: State That’s Been Deep Red Fo...,Republicans have long held the Deep South in t...,News,"August 5, 2016",0
10973,BOMBSHELL COMEY ADMISSION: FBI Found Email Ind...,After the release of tell-all book Shattered ...,politics,"May 4, 2017",0
17081,"Australian senate rejects proposed visa, citiz...",SYDNEY (Reuters) - Australia s lawmakers have ...,worldnews,"October 19, 2017",1
16337,WILL SEXTING By Anthony Weiner Finally Bring D...,WASHINGTON Federal law enforcement officials...,Government News,"Oct 28, 2016",0


In [ ]:
data = data.drop(["title", "subject", "date"], axis=1)

In [ ]:
data.reset_index(inplace=True)

In [ ]:
data.reset_index(inplace=True)

In [ ]:
data.sample(5)

,level_0,index,text,class
39093,39093,15612,SYDNEY (Reuters) - Australian Prime Minister M...,1
42241,42241,18760,KUALA LUMPUR (Reuters) - Organizers of pop con...,1
43296,43296,19815,(Reuters) - Gasoline and diesel prices rose sh...,1
11321,11321,11321,I would love to see 237 votes on the House si...,0
8895,8895,8895,President Obama got real and raw with CNN s An...,0


In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", " ", text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

<>:3: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\w'
C:\Users\sachi\AppData\Local\Temp\ipykernel_20916\2400800703.py:3: SyntaxWarning: invalid escape sequence '\['
  text = re.sub('\[.*?\]', '', text)
C:\Users\sachi\AppData\Local\Temp\ipykernel_20916\2400800703.py:5: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub('https?://\S+|www\.\S+', '', text)
C:\Users\sachi\AppData\Local\Temp\ipykernel_20916\2400800703.py:9: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub('\w*\d\w*', '', text)


In [ ]:
data["text"] = data["text"].apply(clean_text)

In [ ]:
x=data["text"]
y=data["class"]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.25, random_state=42)

In [ ]:
vectorizer = TfidfVectorizer()
x_vtrain = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)

In [ ]:
lr = LogisticRegression()
lr.fit(x_vtrain, ytrain)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [ ]:
prediction =  lr.predict(xv_test)
lr.score(xv_test, ytest)


0.9858351893095768

In [ ]:
print(classification_report(ytest, prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [ ]:
joblib.dump(vectorizer, "vectorizer.jb")
joblib.dump(lr, "model.jb")

['model.jb']

: 